In [20]:
import pandas as pd
import datetime as dt

In [21]:
dt.time(hour=7)

datetime.time(7, 0)

In [22]:
dfr = pd.read_excel("./data/clock_inout.xlsx")

In [23]:
dfr.head()

,รหัสที่เครื่อง,รหัสพนักงาน,ชื่อ-นามสกุล,แผนก,Date,1,2,3
0,1,NaN,รุ้ง,[Start Department],1/2/2567,8:52,18:18,NaN
1,1,NaN,รุ้ง,[Start Department],2/2/2567,9:00,18:14,NaN
2,1,NaN,รุ้ง,[Start Department],3/2/2567,9:23,17:03,NaN
3,1,NaN,รุ้ง,[Start Department],4/2/2567,NaN,NaN,NaN
4,1,NaN,รุ้ง,[Start Department],5/2/2567,8:46,17:26,NaN


In [24]:
dfr.duplicated(subset=["ชื่อ-นามสกุล", "Date", 1, 2, 3]).sum()

0

In [25]:
filtNull = dfr[[1, 2, 3]].isnull().all(axis=1)
dfr = dfr[~filtNull]

In [26]:
def parseDate(dateStr):
    sp = dateStr.split("/")
    day = sp[0]
    month = sp[1]
    year = int(sp[2]) - 543
    return pd.to_datetime(f"{year}/{month}/{day}", format="%Y/%m/%d")


dfr["Date"] = dfr["Date"].apply(parseDate)

In [27]:
# Convert to time
dfr[1] = pd.to_datetime(dfr[1], format="%H:%M").dt.time
dfr[2] = pd.to_datetime(dfr[2], format="%H:%M").dt.time
dfr[3] = pd.to_datetime(dfr[3], format="%H:%M").dt.time


In [28]:
dfr.head()

,รหัสที่เครื่อง,รหัสพนักงาน,ชื่อ-นามสกุล,แผนก,Date,1,2,3
0,1,NaN,รุ้ง,[Start Department],2024-02-01,08:52:00,18:18:00,NaT
1,1,NaN,รุ้ง,[Start Department],2024-02-02,09:00:00,18:14:00,NaT
2,1,NaN,รุ้ง,[Start Department],2024-02-03,09:23:00,17:03:00,NaT
4,1,NaN,รุ้ง,[Start Department],2024-02-05,08:46:00,17:26:00,NaT
5,1,NaN,รุ้ง,[Start Department],2024-02-06,08:52:00,17:56:00,NaT


In [29]:
# Remove rows with hours outside acceptable range
def checkTimeOutsideRange(sr):
    return (sr < dt.time(hour=6)) | (sr > dt.time(hour=22))


filt1 = checkTimeOutsideRange(dfr[1])
filt2 = checkTimeOutsideRange(dfr[2])
filt3 = checkTimeOutsideRange(dfr[3])

filtOutsideRange = filt1 | filt2 | filt3
dfr[filtOutsideRange]

,รหัสที่เครื่อง,รหัสพนักงาน,ชื่อ-นามสกุล,แผนก,Date,1,2,3
39,2,NaN,แปง,[Start Department],2024-02-11,00:00:00,NaT,NaT
126,5,NaN,5,[Start Department],2024-02-11,00:35:00,NaT,NaT
329,16,NaN,เมือง,[Start Department],2024-02-11,00:00:00,NaT,NaT
416,19,NaN,ไอซ์,[Start Department],2024-02-11,00:39:00,NaT,NaT


In [30]:
dfr = dfr[~filtOutsideRange]

In [31]:
dfr["incompleteInOut"] = False
filtOneCheckIn = (~dfr[[1, 2, 3]].isnull()).sum(axis=1) == 1
dfr.loc[filtOneCheckIn, "incompleteInOut"] = True

In [36]:
def addTimeForIncompleteCheckInOut(sr):
    time = sr[1]
    if time < dt.time(hour=13):
        sr[2] = dt.time(hour=18)
    else:
        sr[2] = dt.time(hour=9)
    return sr

filtIIO = dfr["incompleteInOut"] 
dfr.loc[filtIIO,:] = dfr.loc[filtIIO,:].apply(addTimeForIncompleteCheckInOut, axis=1)

In [38]:
dfr.loc[filtIIO].head()

,รหัสที่เครื่อง,รหัสพนักงาน,ชื่อ-นามสกุล,แผนก,Date,1,2,3,incompleteInOut
9,1,NaN,รุ้ง,[Start Department],2024-02-10,09:27:00,18:00:00,NaT,True
23,1,NaN,รุ้ง,[Start Department],2024-02-24,09:41:00,18:00:00,NaT,True
28,1,NaN,รุ้ง,[Start Department],2024-02-29,08:04:00,18:00:00,NaT,True
38,2,NaN,แปง,[Start Department],2024-02-10,07:50:00,18:00:00,NaT,True
57,2,NaN,แปง,[Start Department],2024-02-29,07:49:00,18:00:00,NaT,True


In [33]:
# # Parse datetime from string
# def parseDateTime(dateStr, timeStr):
#     sp = dateStr.split("/")
#     day = sp[0]
#     month = sp[1]
#     year = int(sp[2]) - 543
#     return pd.to_datetime(f"{year}/{month}/{day} {timeStr}", format="%Y/%m/%d %H:%M")


# def parseDate(dateStr):
#     sp = dateStr.split("/")
#     day = sp[0]
#     month = sp[1]
#     year = int(sp[2]) - 543
#     return pd.to_datetime(f"{year}/{month}/{day}", format="%Y/%m/%d")


# print(parseDateTime("1/2/2567", "8:52"))
# print(parseDate("1/2/2567"))

2024-02-01 08:52:00
2024-02-01 00:00:00


In [39]:
def calculateInOut(row):
    times = row.loc[[1,2,3]].dropna()
    res = times.agg(["min", "max"])
    return pd.concat([row, res])

dft = dfr.apply(calculateInOut, axis=1)
dft = dft.rename(columns={"min": "in", "max": "out"})

In [40]:
dft.head()

,รหัสที่เครื่อง,รหัสพนักงาน,ชื่อ-นามสกุล,แผนก,Date,1,2,3,incompleteInOut,in,out
0,1,NaN,รุ้ง,[Start Department],2024-02-01,08:52:00,18:18:00,NaT,False,08:52:00,18:18:00
1,1,NaN,รุ้ง,[Start Department],2024-02-02,09:00:00,18:14:00,NaT,False,09:00:00,18:14:00
2,1,NaN,รุ้ง,[Start Department],2024-02-03,09:23:00,17:03:00,NaT,False,09:23:00,17:03:00
4,1,NaN,รุ้ง,[Start Department],2024-02-05,08:46:00,17:26:00,NaT,False,08:46:00,17:26:00
5,1,NaN,รุ้ง,[Start Department],2024-02-06,08:52:00,17:56:00,NaT,False,08:52:00,17:56:00


In [152]:
dft["isInLate"] = dft["in"] > dft["date"] + pd.Timedelta(hours=9)
dft["isOutEarly"] = dft["out"] < dft["date"] + pd.Timedelta(hours=18)
dft.head()

,in,out,date,name,id,isInLate,isOutEarly
0,2024-02-01 08:52:00,2024-02-01 18:18:00,2024-02-01,รุ้ง,1,False,False
1,2024-02-02 09:00:00,2024-02-02 18:14:00,2024-02-02,รุ้ง,1,False,False
2,2024-02-03 09:23:00,2024-02-03 17:03:00,2024-02-03,รุ้ง,1,True,True
4,2024-02-05 08:46:00,2024-02-05 17:26:00,2024-02-05,รุ้ง,1,False,True
5,2024-02-06 08:52:00,2024-02-06 17:56:00,2024-02-06,รุ้ง,1,False,True


In [153]:
dft["inLateMin"] = (
    dft["in"] - dft["date"] - pd.Timedelta(hours=9)
).dt.total_seconds() / 60
dft["outEarlyMin"] = (
    dft["date"] + pd.Timedelta(hours=18) - dft["out"]
).dt.total_seconds() / 60


In [154]:
def makeZeroIfNegative(val):
    if val < 0:
        return 0
    return val


dft["inLateMin"] = dft["inLateMin"].apply(makeZeroIfNegative)
dft["outEarlyMin"] = dft["outEarlyMin"].apply(makeZeroIfNegative)

dft.head()

,in,out,date,name,id,isInLate,isOutEarly,inLateMin,outEarlyMin
0,2024-02-01 08:52:00,2024-02-01 18:18:00,2024-02-01,รุ้ง,1,False,False,0.0,0.0
1,2024-02-02 09:00:00,2024-02-02 18:14:00,2024-02-02,รุ้ง,1,False,False,0.0,0.0
2,2024-02-03 09:23:00,2024-02-03 17:03:00,2024-02-03,รุ้ง,1,True,True,23.0,57.0
4,2024-02-05 08:46:00,2024-02-05 17:26:00,2024-02-05,รุ้ง,1,False,True,0.0,34.0
5,2024-02-06 08:52:00,2024-02-06 17:56:00,2024-02-06,รุ้ง,1,False,True,0.0,4.0


In [155]:
def formatTorDrinkExcelP2(row):
    datas = []
    for i in ["IN", "OUT"]:
        data = {
            "attendance_device_id": row["name"],
            "log_type": i,
            "time": row[i].strftime("%Y/%m/%d %X"),
        }
        datas.append(data)
    return pd.DataFrame.from_records(datas)


temp = dft.iloc[:, :].apply(formatTorDrinkExcelP2, axis=1)
df2 = pd.concat(temp.values).reset_index(drop=True)
df2

KeyError: 'IN'

In [67]:
sr = df2.iloc[0, :].to_dict()

In [68]:
{**df2.iloc[0, :].to_dict()}

{'attendance_device_id': 'รุ้ง',
 'log_type': 'IN',
 'time': '2024/02/01 08:52:00'}